In [1]:
from test_case_agent import testcase_agent
from optimization_agent import optimization_suggestor_agent

## Input

In [2]:
code = """def calculate_factorials_up_to_number(n):
    if n < 0:
        raise ValueError("Input must be a non-negative integer.")

    factorials = [1]  # Initialize with the factorial of 0

    factorial = 1
    for i in range(1, n + 1):
        factorial *= i
        factorials.append(factorial)

    return factorials"""

## Initialize Testcase Agent

In [3]:
agent = testcase_agent(code = code,
                    #    test_case_list =["calculate_factorials_up_to_number(100000)"]
                       )

In [4]:
vas = agent.run_flow()

-----test case generation---
Worst case
```python
calculate_factorials_up_to_number(1000)
```

Medium case
```python
calculate_factorials_up_to_number(10)
```

Best case
```python
calculate_factorials_up_to_number(0)
```


-----test case extraction---
calculate_factorials_up_to_number(1000)
--------
calculate_factorials_up_to_number(10)
--------
calculate_factorials_up_to_number(0)
--------
None


-----test case merger---
import time
import sys
import os
import psutil

def memory_usage_psutil():
    # return the memory usage in MB
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss

def test_case():
    start_time = time.time()
    start_memory = memory_usage_psutil()

    # Call the function here
    calculate_factorials_up_to_number(1000)
    
    end_time = time.time()
    end_memory = memory_usage_psutil()

    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Memory used: {end_memory - start_memory} B")

def calcul

In [5]:
from pprint import pprint 
pprint(vas)

{'response': [{'Memory used': 524288.0, 'Time taken': 0.0005857944488525391},
              {'Memory used': 0.0, 'Time taken': 0.00013208389282226562},
              {'Memory used': 0.0, 'Time taken': 0.0001239776611328125}],
 'testcase': ['calculate_factorials_up_to_number(1000)',
              'calculate_factorials_up_to_number(10)',
              'calculate_factorials_up_to_number(0)']}


------

## Optimize Code

In [8]:
op_agent = optimization_suggestor_agent(code= code,
                                        execution_obj= vas)

In [9]:
vas = op_agent.run_flow()

----raw optimizations----
The code you provided calculates and returns a list of factorials up to a given number 'n'. Here are some suggestions for optimizing the code:

1. Error Handling: The code checks if the input 'n' is a non-negative integer and raises a `ValueError` if it's not. It would be better to use a built-in function, such as `isinstance(n, int)`, to check if 'n' is an integer and `n >= 0` to check if it's non-negative. This would simplify the error handling and remove the need to raise an exception explicitly.

2. Use List Comprehension: Instead of initializing an empty list `factorials` and then appending to it in each iteration, you can use list comprehension to generate the list directly. This would eliminate the need for the explicit loop and the `append()` method.

3. Calculate the Factorial Incrementally: The code currently multiplies the factorial with 'i' in each iteration. Instead, you can calculate the next factorial by multiplying the previous factorial with '

In [10]:
print(vas)

def calculate_factorials_up_to_number(n):
    if not isinstance(n, int) or n < 0:
        raise ValueError("Input must be a non-negative integer.")

    factorials = [1] * (n + 1)

    for i in range(2, n + 1):
        factorials[i] = factorials[i - 1] * i

    return factorials


------------------

In [23]:
extracted_blocks = """def calculate_factorials_up_to_number(n):
    if not isinstance(n, int) or n < 0:
        raise ValueError("Input must be a non-negative integer.")

    factorials = {}

    def factorial(k):
        if k in factorials:
            return factorials[k]
        elif k == 0:
            return 1
        else:
            result = k * factorial(k - 1)
            factorials[k] = result
            return result

    return [factorial(i) for i in range(n + 1)]"""

## Optimized Code Testcases

In [24]:
op_code_test = testcase_agent(
    code = extracted_blocks)

In [25]:
op_code_test.gen_test_case()
op_code_test.extract_test_cases()
op_code_test.join_test_case()
op_code_test.run_test_case()
vas = op_code_test.extract_info_tg()

calculate_factorials_up_to_number(1000)
--------
calculate_factorials_up_to_number(10)
--------
calculate_factorials_up_to_number(0)
--------


In [26]:
print(vas)

[{'Time taken': 0.0008258819580078125, 'Memory used': 655360.0}, {'Time taken': 0.00017380714416503906, 'Memory used': 0.0}, {'Time taken': 0.00018930435180664062, 'Memory used': 0.0}]


----

## Profiler

In [1]:
from profiler_agent import profiler

In [2]:
code = """def do_stuff(numbers):
    s = sum(numbers)
    prod = 1
    for n in numbers:
        prod *= n
    return s, prod"""
testcase = "do_stuff([1, 2, 3, 4, 50000000000000000000000])"

In [3]:
p_agent = profiler(
    code= code,
    testcase= testcase
                   )

In [4]:
vas = p_agent.run_flow()

command ran
dict_keys(['s = sum(numbers)', 'prod = 1', 'for n in numbers:', 'prod *= n', 'return s, prod'])
def do_stuff(numbers):
    s = sum(numbers) #percent_time: 32.2%
    prod = 1 #percent_time: 5.0%
    for n in numbers: #percent_time: 28.8%
        prod *= n #percent_time: 28.8%
    return s, prod #percent_time: 5.2%


In [5]:
print(vas)

def do_stuff(numbers):
    s = sum(numbers) #percent_time: 32.2%
    prod = 1 #percent_time: 5.0%
    for n in numbers: #percent_time: 28.8%
        prod *= n #percent_time: 28.8%
    return s, prod #percent_time: 5.2%


---------

## Overseer

In [1]:
dataset_raw = open("dataset/single_input_functions.txt").read()

import re
dataset_mutated = re.sub(r'#\$%\d+\$%#', '', dataset_raw)

dataset_mutated = dataset_mutated.split("\n\n")
dataset = [item for item in dataset_mutated if item.strip() != ""]
print(len(dataset))
# print(*dataset_mutated, sep= "\n---\n----")
 

50


In [2]:
from overseer import flow_master

In [2]:
# code = """def monte_carlo_pi(iterations):
#     inside_circle = 0
#     import random
#     for _ in range(iterations):
#         x, y = random.random(), random.random()
#         if x**2 + y**2 <= 1:
#             inside_circle += 1
#     return inside_circle"""

In [26]:
# code = """def compress_list_values(list_values):
#     compressed_list = []
#     for value in list_values:
#         if value not in compressed_list:
#             compressed_list.append(value)
#     return compressed_list"""

In [3]:
dataset_response = {}

In [4]:
from tqdm import tqdm

In [ ]:
counter = 0
value_error = 0 # no code found
r_err = 0 

while counter < 5:
    try:
        for idx, instance in tqdm(enumerate(dataset)):
            flow = flow_master(code= instance)

            parent, t_m = flow.start()

            for key in t_m.keys():
                print(key)
                print(t_m[key][0])
            dataset_response[instance] = (parent, t_m)
            print("____________________")
            print(("____________________" + '\n') * 10, end='')

        counter += 1
    except ValueError as exp:
        value_error += 1
        print(exp)
        continue
    except Exception as exp:
        r_err += 1
        print(exp)
        continue



In [11]:
from pprint import pprint 
pprint(dataset_response)

{'\ndef bubble_sort(lst):\n    n = len(lst)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if lst[j] > lst[j+1] :\n                lst[j], lst[j+1] = lst[j+1], lst[j]\n    return lst': ({'original code': {'response': [{'Memory used': 0.0,
                                                                                                                                                                                                                                                    'Time taken': 0.00014019012451171875},
                                                                                                                                                                                                                                                   {'Memory used': 0.0,
                                                                                                                                                                                             

original code
{'Time taken': 0.00014162063598632812, 'Memory used': 0.0}
phase 1
{'Time taken': 0.00012922286987304688, 'Memory used': 0.0}
phase 2
{'Time taken': 0.0001227855682373047, 'Memory used': 0.0}


---

## Executor Agent

In [1]:
code = """import math

def calculate_factorials_up_to_number(n):
    assert n >= 0, "Input must be a non-negative integer."

    factorials = [math.factorial(i) for i in range(n + 1)]

    return factorials"""

testcase = ["calculate_factorials_up_to_number(1000)", "calculate_factorials_up_to_number(100)", "calculate_factorials_up_to_number(10)"]

In [2]:
from executor_agent import code_executor

In [3]:
ce = code_executor(
    code = code,
    testcase = testcase
)

In [4]:
ce.run()

[{'Time taken': 0.012521505355834961, 'Memory used': 655360.0},
 {'Time taken': 0.0001900196075439453, 'Memory used': 0.0},
 {'Time taken': 0.00023603439331054688, 'Memory used': 0.0}]

---

## Optimization Agent

In [1]:
text = """The code provided reads a CSV file, adds each row into a list, and then performs some calculations on the data. However, there are a few inefficiencies that can be improved:

1. Use of intermediary "data" list: This list is not necessary and only consumes memory. You can perform the calculations as you read each row from the CSV file.

2. Nested loops: The nested loops (iteration over the data for calculations) increase the computational complexity of the program. This is unnecessary as you can perform the operations within a single loop.

3. Redundant file closure: When using the `with` keyword to open a file, Python automatically closes the file once it is out of scope. So the file closure at the end of the script is redundant.

4. Exception Handling: Instead of catching ValueError for every value, it could be more efficient to use a comprehension that filters out non-numerics before converting them to floats.

Here is a more optimized version of the function:

```python
import csv

def optimized_csv_processing(filename):
    # Initialize results
    results = []

    # Open the CSV file
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        
        # Read data and perform calculations
        for row in reader:
            if len(row) >= 3:
                result = sum(float(value) for value in row if value.replace('.', '', 1).isdigit())
                results.append(result)

    # Return the results
    return results
```"""

In [1]:
exec_obj = {'response': [{'Memory used': 524288.0, 'Time taken': 0.0005857944488525391},
              {'Memory used': 0.0, 'Time taken': 0.00013208389282226562},
              {'Memory used': 0.0, 'Time taken': 0.0001239776611328125}],
 'testcase': ['calculate_factorials_up_to_number(1000)',
              'calculate_factorials_up_to_number(10)',
              'calculate_factorials_up_to_number(0)']}
 
code = """def calculate_factorials_up_to_number(n):
    if n < 0: #percent_time: 29.3%
        raise ValueError("Input must be a non-negative integer.")

    factorials = [1]  # Initialize with the factorial of 0

    factorial = 1 #percent_time: 5.6%
    for i in range(1, n + 1): #percent_time: 44.3%
        factorial *= i
        factorials.append(factorial)

    return factorials #percent_time: 7.7%"""

In [2]:
from optimization_agent import optimization_suggestor_agent

In [3]:
op = optimization_suggestor_agent(code= code,
                                execution_obj = exec_obj)

In [4]:
vas = op.run_flow_phase_1()

In [5]:
from pprint import pprint
pprint(vas)

{'exec_info': [{'Memory used': 0.0, 'Time taken': 0.00017404556274414062},
               {'Memory used': 0.0, 'Time taken': 0.00012826919555664062},
               {'Memory used': 0.0, 'Time taken': 0.0001857280731201172}],
 'optimized_code': '\n'
                   'def calculate_factorials_up_to_number(n):\n'
                   '    factorial = 1\n'
                   '    yield factorial  # The factorial of 0 is 1\n'
                   '    for i in range(1, n + 1):\n'
                   '        factorial *= i\n'
                   '        yield factorial\n',
 'suggestions': 'The provided code calculates the list of factorials of all '
                'numbers up to a given input number. This is done in a single '
                'loop, which is fairly efficient time complexity-wise. '
                'However, there are a few places where the function could be '
                'further optimized:\n'
                '\n'
                '1) Input validation: The error checking f

In [6]:
vas = op.run_flow_phase_2()

command ran
dict_keys([])

def calculate_factorials_up_to_number(n):
    factorial = 1
    yield factorial  # The factorial of 0 is 1
    for i in range(1, n + 1):
        factorial *= i
        yield factorial



In [7]:
print(vas)

The optimized function you've used is effectively yielding factorials one by one, which drastically reduces memory usage to almost 0 in all cases. The operation is carried out in constant time complexity O(n). So in terms of performance, this is already excellent.

However, there are still potential improvements, mostly applying to readability, error-handling and input constraints.

1) Exception Handling: Negative numbers and non-integer inputs are not valid and may cause the function to behave unexpectedly. Adding exception or error handling to deal with this could be beneficial.

2) Function Docstrings: Adding docstrings to the function to explain what it does, its parameters, its return type and any exceptions it can raise. This makes it easier for other developers to understand and use the function.

Here is the code after applying these changes:

```python
def calculate_factorials_up_to_number(n):
    """
    Function to calculate the factorial of numbers up to n and yield each fa

In [8]:
pprint(op.phase_2_tuples)

[{'content': 'You are a senior python developer. You will be given an input '
             'code with its execution time and memory used. \n'
             'You will critically analyse the code and optimize the code all '
             'while keeping the name, inputs and output structure the same.\n',
  'role': 'system'},
 {'content': ' ```python\n'
             'def do_stuff(numbers):\n'
             '    s = sum(numbers) #percent_time: 32.2%\n'
             '    prod = 1 #percent_time: 5.0%\n'
             '    for n in numbers: #percent_time: 28.8%\n'
             '        prod *= n #percent_time: 28.8%\n'
             '    return s, prod #percent_time: 5.2%```\n'
             'Worst Case;\n'
             'Time taken: 0.00013065338134765625\n'
             'Memory used: 0.0\n'
             '\n'
             'Medium Case;\n'
             'Time taken: 0.0002562999725341797\n'
             'Memory used: 0.0\n'
             '\n'
             'Best Case;\n'
             'Time taken: 0.0002

# --test

In [8]:
l = [1,2,'3',4]

def g(x):
    if type(x) != "str":
        raise ValueError("exp")
    
    else:
        print(x)

try:
    for i in l:
        print(g(i))
except ValueError as e:
    print(e)

exp


In [6]:
type("s")

str

In [5]:
l = [1, 2, '3', 4]

def g(x):
    if type(x) != str:
        raise ValueError("exp")
    else:
        print(x)

from tqdm import tqdm
import time

for idx, i in enumerate(tqdm(l)):
    try:
        time.sleep(5)
        g(i)
    except ValueError as e:
        print(e)
        continue


 25%|██▌       | 1/4 [00:05<00:15,  5.00s/it]

exp


 50%|█████     | 2/4 [00:10<00:10,  5.00s/it]

exp


 75%|███████▌  | 3/4 [00:15<00:05,  5.00s/it]

3


100%|██████████| 4/4 [00:20<00:00,  5.00s/it]

exp


------

## Final Json Parser

In [1]:
s_j = {
  "\ndef bubble_sort(lst):\n    n = len(lst)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if lst[j] > lst[j+1] :\n                lst[j], lst[j+1] = lst[j+1], lst[j]\n    return lst": [
    {
      "original code": {
        "response": [
          {
            "Time taken": 0.0005500316619873047,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0007779598236083984,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.00035762786865234375,
            "Memory used": 0.0
          }
        ],
        "testcase": [
          "bubble_sort([9, 8, 7, 6, 5, 4, 3, 2, 1])",
          "bubble_sort([5, 2, 8, 1, 3])",
          "bubble_sort([1, 2, 3, 4, 5])"
        ]
      },
      "phase 1": {
        "optimized_code": "\ndef bubble_sort(lst):\n    n = len(lst)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            # Only perform the swap operation if necessary\n            lst[j], lst[j+1] = min(lst[j], lst[j+1]), max(lst[j], lst[j+1])\n    return lst\n",
        "suggestions": "The provided code performs a bubble sort on the given list. Bubble sort is a simple but inefficient sorting algorithm. It continually swaps adjacent elements if they are not in the correct order. The algorithm stops when it manages to go through the entire list without having to perform a swap. \n\nEven though bubble sort has O(n^2) computational complexity, which is not considered efficient in comparison to other sorting algorithms, we don't need to replace it outright if the list being sorted is relatively small or nearly sorted. Indeed, in its best case, when the list is already sorted, bubble sort has a linear computational complexity O(n).\n\nHowever, the bubble sort algorithm can be optimized with a small tweak. From the given profile, one can see that almost 25.9% of the time is spent on checking whether `lst[j] > lst[j+1]`. But in python, you can make a swap operation only if it's necessary (in one line) which should make the code much faster and cleaner. Here's how:\n\n",
        "exec_info": [
          {
            "Time taken": 0.0005764961242675781,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.000308990478515625,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0008366107940673828,
            "Memory used": 0.0
          }
        ]
      },
      "phase 2": {
        "optimized_code": "\ndef bubble_sort(lst):\n    n = len(lst)\n    for i in range(n):\n        swapped = False\n        for j in range(0, n-i-1):\n            if lst[j] > lst[j+1]:\n                lst[j], lst[j+1] = lst[j+1], lst[j]\n                swapped = True\n        if not swapped:\n            break\n    return lst\n",
        "suggestions": "The code still remains inefficient with the time complexity of O(n^2), and since the newly suggested swapping technique which removes explicit if condition ends up calling built-in functions min and max for every iteration, it might have increased the execution time in certain situations. \n\nInstead, we could save additional computation by adding a flag to check if there was any swapping in inner loop. If there was no swapping, it means the list is already sorted and we can break from the loop. This will speed up the best case scenario to O(n).\n\nHere's how you can implement this:\n\n",
        "exec_info": [
          {
            "Time taken": 0.0005695819854736328,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0002453327178955078,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.00046253204345703125,
            "Memory used": 0.0
          }
        ]
      }
    },
    {
      "original code": [
        {
          "Time taken": 0.0005500316619873047,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0007779598236083984,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.00035762786865234375,
          "Memory used": 0.0
        }
      ],
      "phase 1": [
        {
          "Time taken": 0.0005764961242675781,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.000308990478515625,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0008366107940673828,
          "Memory used": 0.0
        }
      ],
      "phase 2": [
        {
          "Time taken": 0.0005695819854736328,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0002453327178955078,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.00046253204345703125,
          "Memory used": 0.0
        }
      ]
    }
  ],
  "\ndef quick_sort(lst):\n    if len(lst) <= 1:\n        return lst\n    pivot = lst[len(lst) // 2]\n    left = [x for x in lst if x < pivot]\n    middle = [x for x in lst if x == pivot]\n    right = [x for x in lst if x > pivot]\n    return quick_sort(left) + middle + quick_sort(right)": [
    {
      "original code": {
        "response": [
          {
            "Time taken": 0.00063323974609375,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0005657672882080078,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.00015044212341308594,
            "Memory used": 0.0
          }
        ],
        "testcase": [
          "quick_sort([1000, 999, 998, 997, 996, 995, 994, 993, 992, 991])",
          "quick_sort([5, 2, 8, 1, 9, 3, 7, 4, 6])",
          "quick_sort([1, 2, 3, 4, 5, 6, 7, 8, 9])"
        ]
      },
      "phase 1": {
        "optimized_code": "\ndef quick_sort(lst):\n    if len(lst) <= 1: #percent_time: 16.1%\n        return lst\n    else:\n        left, middle, right = [], [], [] # Initialize all lists at once\n        pivot_index = len(lst) // 2\n        pivot = sorted([lst[0], lst[pivot_index], lst[-1]])[1] # Take median of first, middle and last elements as pivot\n        for x in lst:\n            if x < pivot:\n                left.append(x)\n            elif x == pivot:\n                middle.append(x)\n            else:  \n                right.append(x) \n        return quick_sort(left) + middle + quick_sort(right) \n",
        "suggestions": "The provided code implements the quick sort algorithm in Python. It takes in a list and returns a sorted version of it. The function iterates over the list multiple times, creating three new lists in each iteration: one for elements smaller than the pivot (left), one for elements equal to the pivot (middle), and one for elements greater than the pivot (right). This can be inefficient, especially for large lists.\n\nThe key points for optimization are:\n\n1. Avoiding multiple iterations over the list by merging the operations of dividing elements into 'left', 'middle', and 'right' lists.\n\n2. Reducing memory usage by avoiding the creation of extra lists.\n\n3. If the list is sorted or nearly sorted, the time complexity of this quick sort function would be O(n^2). Choosing a better pivot can improve performance in these cases.\n\nOptimized version of the function:\n\nHere is more optimized version of `quick_sort` function which uses list comprehension only once and also picks pivot as a median of the first, middle and last elements which can improve performance for sorted or nearly sorted lists.\n\n",
        "exec_info": [
          {
            "Time taken": 0.0006146430969238281,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0007653236389160156,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.00020265579223632812,
            "Memory used": 0.0
          }
        ]
      },
      "phase 2": {
        "optimized_code": "\ndef quick_sort(lst):\n    if len(lst) <= 1: \n        return lst \n    else:\n        pivot = lst.pop()  # Use the last item as the pivot\n        less_than, greater_than = [], []  # Create two lists for items less than and greater than the pivot\n\n        for x in lst:  \n            if x <= pivot: \n                less_than.append(x)\n            else:  \n                greater_than.append(x)  \n\n        return quick_sort(less_than) + [pivot] + quick_sort(greater_than) \n",
        "suggestions": "Let's analyze the areas where time is taken:\n\nChoosing the pivot: In the current implementation, the code chooses the pivot as the median of the first, middle, and last elements of the list. To find this median, it creates a new list containing these three elements, sorts it, and then indexes it \u2013 operations that take quite some time.\n\nPartitioning the list: The code currently partitions the list into three new lists: one for elements less than the pivot, one for elements equal to the pivot, and one for elements greater than the pivot. Each partition involves a separate pass through the original list, meaning that this step requires three full iterations over the list.\n\nThe creation of these new lists requires considerable time and memory.\n\nTo address these issues, we will use the Lomuto partition scheme which partitions in-place and select the pivot in an efficient manner.\n\n",
        "exec_info": [
          {
            "Time taken": 0.0005640983581542969,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0006017684936523438,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0006911754608154297,
            "Memory used": 131072.0
          }
        ]
      }
    },
    {
      "original code": [
        {
          "Time taken": 0.00063323974609375,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0005657672882080078,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.00015044212341308594,
          "Memory used": 0.0
        }
      ],
      "phase 1": [
        {
          "Time taken": 0.0006146430969238281,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0007653236389160156,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.00020265579223632812,
          "Memory used": 0.0
        }
      ],
      "phase 2": [
        {
          "Time taken": 0.0005640983581542969,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0006017684936523438,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0006911754608154297,
          "Memory used": 131072.0
        }
      ]
    }
  ],
  "def insertion_sort(lst):\n    for i in range(1, len(lst)):\n        key = lst[i]\n        j = i-1\n        while j >=0 and key < lst[j] :\n                lst[j+1] = lst[j]\n                j -= 1\n        lst[j+1] = key\n    return lst": [
    {
      "original code": {
        "response": [
          {
            "Time taken": 0.0005545616149902344,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0007030963897705078,
            "Memory used": 131072.0
          },
          {
            "Time taken": 0.0001728534698486328,
            "Memory used": 0.0
          }
        ],
        "testcase": [
          "insertion_sort([9, 8, 7, 6, 5, 4, 3, 2, 1])",
          "insertion_sort([5, 2, 8, 3, 1, 6, 4])",
          "insertion_sort([1, 2, 3, 4, 5, 6, 7, 8, 9])"
        ]
      },
      "phase 1": {
        "optimized_code": "\ndef insertion_sort(lst):\n    for i in range(1, len(lst)):\n        key = lst[i]\n        j = i - 1\n        # Shift elements of lst[0..i-1], that are greater than key, to one position ahead of their current position\n        while j >= 0 and key < lst[j]:\n            lst[j + 1] = lst[j]\n            j -= 1\n        lst[j + 1] = key\n    return lst\n",
        "suggestions": "The given function is implementing the Insertion Sort algorithm. This algorithm works by dividing the list into sorted and unsorted regions. The sorted region begins with the first element only, and with each iteration, one element from the unsorted region is taken, and inserted into the correct position in the sorted region. While this algorithm is very simple, it is not very efficient with larger data sets - it has time complexity of O(n^2), which means it will start to slow down significantly as the size of the input list increases.\n\nIn this specific function, most of the time is spent in the inner while loop where the insertion into the correct position in the sorted region is done. This accounts for 20.9% of the time taken. This 'insertion' is done by shifting the elements of the sorted region that are larger than the key value one position to the right. The key value is then placed in its correct position in the sorted region.\n\nCurrently, the way this function is written cannot be optimized further without altering the algorithm or the function signature. The percentage values indicate the distribution of time spent on individual lines of code, which is consistent with the nature of the insertion sort algorithm - the shifting operation in the sorted region (which happens within the inner while loop) is quite time-consuming, especially for larger lists. If you want to avoid the time consumption of insertion sort, consider using more efficient sorting algorithm such as quicksort, mergesort, or heapsort instead. \n\nHere is the same function written with minor improvements for readability:\n\n",
        "exec_info": [
          {
            "Time taken": 0.0005462169647216797,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0005142688751220703,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0008046627044677734,
            "Memory used": 0.0
          }
        ]
      },
      "phase 2": {
        "optimized_code": "\ndef insertion_sort(lst):\n    return sorted(lst)\n",
        "suggestions": "The given code is an implementation of the insertion sort algorithm. In its essence, the insertion sort algorithm is inefficient, as it's characterized by a worst-case and average time complexity of O(n\u00b2), which makes it inefficient for large data sets. Therefore, the written code that correctly fulfils the algorithm cannot be substantially optimized without changing the fundamental algorithm.\n\nHowever, if you are working with a small dataset, this won't really be a problem and the code is already quite optimized. Additionally, insertion sort has the benefit of being an \"in-place\" sort, meaning it does not require additional space as it only creates a sorted list by reordering the elements within the list. This is evident as the memory use in all three cases does not increase.\n\nIn Python, we can always sort our lists using the built-it sorted(list) function. It implements the Timsort algorithm, which is a hybrid sorting algorithm derived from merge sort and insertion sort designed to perform well on many kinds of real-world data.\n \nIf you strictly need to decrease execution time and are not bound to use an algorithm, you could simply substitute your insertion sort function with a call to Python's built-in sorting function, it would look like this:\n\n",
        "exec_info": [
          {
            "Time taken": 0.0005640983581542969,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0005166530609130859,
            "Memory used": 0.0
          },
          {
            "Time taken": 0.0005168914794921875,
            "Memory used": 0.0
          }
        ]
      }
    },
    {
      "original code": [
        {
          "Time taken": 0.0005545616149902344,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0007030963897705078,
          "Memory used": 131072.0
        },
        {
          "Time taken": 0.0001728534698486328,
          "Memory used": 0.0
        }
      ],
      "phase 1": [
        {
          "Time taken": 0.0005462169647216797,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0005142688751220703,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0008046627044677734,
          "Memory used": 0.0
        }
      ],
      "phase 2": [
        {
          "Time taken": 0.0005640983581542969,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0005166530609130859,
          "Memory used": 0.0
        },
        {
          "Time taken": 0.0005168914794921875,
          "Memory used": 0.0
        }
      ]
    }
  ]
}

In [9]:
k = {'original code': [{'Memory used': 0.0, 'Time taken': 0.0005500316619873047},
                   {'Memory used': 0.0, 'Time taken': 0.0007779598236083984},
                   {'Memory used': 0.0, 'Time taken': 0.00035762786865234375}],
 'phase 1': [{'Memory used': 0.0, 'Time taken': 0.0005764961242675781},
             {'Memory used': 0.0, 'Time taken': 0.000308990478515625},
             {'Memory used': 0.0, 'Time taken': 0.0008366107940673828}],
 'phase 2': [{'Memory used': 0.0, 'Time taken': 0.0005695819854736328},
             {'Memory used': 0.0, 'Time taken': 0.0002453327178955078},
             {'Memory used': 0.0, 'Time taken': 0.00046253204345703125}]}

In [8]:
from pprint import pprint

In [13]:
# def best_phase(data):
#     sum_times = {phase: sum([i['Time taken'] for i in results]) for phase, results in data.items()}
#     best = min(sum_times, key=sum_times.get)
#     return best

In [14]:
# def best_phase(data):

#     # Calculate the total time for each phase
#     sum_times = {phase: sum([i['Time taken'] for i in results]) for phase, results in data.items()}
    
#     # Calculate the total time across all phases
#     total_time = sum(sum_times.values())

#     # Find the phase with the minimum total time
#     best = min(sum_times, key=sum_times.get)

#     # Calculate the percentage of total time for the best phase
#     best_percentage = (sum_times[best] / total_time) * 100

#     # Calculate the percentage of total time for each phase except the best one
#     other_phases = [p for p in sum_times.keys() if p != best]
#     other_phases_percentage = [((sum_times[p] / total_time) * 100) for p in other_phases]
    
#     # Calculate the difference in percentage between the best phase and the sum of other phases
#     better_by_percentage = sum(other_phases_percentage) - best_percentage

#     return best, better_by_percentage 

In [1]:
def best_phase(data):

    # Calculate the total time for each phase
    sum_times = {phase: sum([i['Time taken'] for i in results]) for phase, results in data.items()}

    # Task 1: Find the phase with the minimum total time
    best = min(sum_times, key=sum_times.get)

    # Task 2: Calculate the percentage decrease of time of phase 1 and phase 2 from original code
    original_time = sum_times['original code']
    phase_1_decrease_percentage = ((original_time - sum_times['phase 1']) / original_time) * 100 if 'phase 1' in sum_times else 0
    phase_2_decrease_percentage = ((original_time - sum_times['phase 2']) / original_time) * 100 if 'phase 2' in sum_times else 0

    return best, (phase_1_decrease_percentage, phase_2_decrease_percentage)


In [2]:
import json

In [3]:
s_j = json.load(open('data.json'))

In [4]:
l = []
new_ds = {}
for instance in s_j:

    try:

        vas = best_phase(s_j[instance][1])
        print(vas)
        l.append(vas)
        new_ds[instance] = {"payload":s_j[instance],
                            "agent_result": vas}


   
   
    except IndexError as e:
        print(e)
        continue
   
   
    except TypeError as e:
        print(e)
        continue

('original code', (-6.755319540884875, -3.806391554561063))
('phase 2', (20.022576548610132, 21.96980386623395))
unsupported operand type(s) for +: 'int' and 'str'
('phase 2', (10.303461455303776, 10.37739138925505))
('original code', (-84.35374149659864, -116.90526957051478))
('phase 1', (82.30084896245637, 82.27837992805941))
('original code', (-10.375, -16.75))
('original code', (-41.24885215794307, -35.05968778696052))
('phase 2', (-103.98955117549275, 50.84303015910711))
('phase 1', (0.26799895454271294, -3.0143348278011217))
list index out of range
('phase 2', (-12.654032463669584, 19.877623863346646))
('original code', (-113.10410635960515, -17.252563968919645))
('original code', (-0.3864447086801427, -16.97384066587396))
list index out of range
('phase 2', (-0.01604524873130037, 0.000720378929505416))
('original code', (-30.89068825910931, -58.98785425101215))
('phase 2', (-5.149253731343284, 1.8905472636815919))
list index out of range
unsupported operand type(s) for +: 'int' 

In [5]:
len(new_ds)

27

In [14]:
with open("interim_ds.json","w") as f:
    json.dump(new_ds, f)

In [6]:
from base_models import base_models_req
from tqdm import tqdm

In [ ]:
final_ds = new_ds

for code_instance in tqdm(new_ds):


    vgpt = base_models_req(code=code_instance,
                       test_case=new_ds[code_instance]["payload"][0]["original code"]["testcase"]
                       )


    vas = vgpt.run_flow()
    final_ds[code_instance]["payload"][0]["vanilla_gpt-3.5"] = vas
    final_ds[code_instance]["payload"][1]["vanilla_gpt-3.5"] = vas["exec_info"]
    pprint(final_ds[code_instance])


with open("final_ds.json", "w") as f:
    json.dump(l, f)




-----

In [23]:
from pprint import pprint
import json
def best_phase_v2(data):

    # Calculate the total time for each phase
    sum_times = {phase: sum([i['Time taken'] for i in results]) for phase, results in data.items()}

    # Task 1: Find the phase with the minimum total time
    best = min(sum_times, key=sum_times.get)

    # Task 2: Calculate the percentage decrease of time of phase 1 and phase 2 from original code
    original_time = sum_times['original code']
    phase_1_decrease_percentage = ((original_time - sum_times['phase 1']) / original_time) * 100 if 'phase 1' in sum_times else 0
    phase_2_decrease_percentage = ((original_time - sum_times['phase 2']) / original_time) * 100 if 'phase 2' in sum_times else 0
    vanilla_gpt_decrease_percentage = ((original_time - sum_times['vanilla_gpt-3.5']) / original_time) * 100 if 'vanilla_gpt-3.5' in sum_times else 0
    return best, (phase_1_decrease_percentage, phase_2_decrease_percentage, vanilla_gpt_decrease_percentage)




In [16]:
s_j = json.load(open('final_ds.json'))

In [26]:
for instance in s_j:
    # pprint(s_j[instance]["payload"][1])
    print(best_phase_v2(s_j[instance]["payload"][1]))
    break

('original code', (-6.755319540884875, -3.806391554561063, -2.5205646339875862))


In [27]:
l = []
new_ds = {}
for instance in s_j:

    try:

        vas = best_phase_v2(s_j[instance]["payload"][1])
        print(vas)
        l.append(vas)
        new_ds[instance] = {"payload":s_j[instance],
                            "agent_result": vas}


   
   
    except IndexError as e:
        print(e)
        continue
   
   
    except TypeError as e:
        print(e)
        continue

('original code', (-6.755319540884875, -3.806391554561063, -2.5205646339875862))
('phase 2', (20.022576548610132, 21.96980386623395, 19.909693805559474))
('phase 2', (10.303461455303776, 10.37739138925505, -1.0322292664895139))
('vanilla_gpt-3.5', (-84.35374149659864, -116.90526957051478, 4.629911666159001))
('phase 1', (82.30084896245637, 82.27837992805941, -22.002491929995813))
('original code', (-10.375, -16.75, -20.78125))
('vanilla_gpt-3.5', (-41.24885215794307, -35.05968778696052, 7.107438016528926))
('phase 2', (-103.98955117549275, 50.84303015910711, 2.968416053194016))
('vanilla_gpt-3.5', (0.26799895454271294, -3.0143348278011217, 2.8418343754397957))
('phase 2', (-12.654032463669584, 19.877623863346646, 12.288603722274157))
('vanilla_gpt-3.5', (-113.10410635960515, -17.252563968919645, 16.290724807351513))
('original code', (-0.3864447086801427, -16.97384066587396, -4.191438763376932))
('phase 2', (-0.01604524873130037, 0.000720378929505416, -146.54029038859983))
('vanilla_gp

In [1]:
l = [('original code', (-6.755319540884875, -3.806391554561063, -2.5205646339875862)),
('phase 2', (20.022576548610132, 21.96980386623395, 19.909693805559474)),
('phase 2', (10.303461455303776, 10.37739138925505, -1.0322292664895139)),
('vanilla_gpt-3.5', (-84.35374149659864, -116.90526957051478, 4.629911666159001)),
('phase 1', (82.30084896245637, 82.27837992805941, -22.002491929995813)),
('original code', (-10.375, -16.75, -20.78125)),
('vanilla_gpt-3.5', (-41.24885215794307, -35.05968778696052, 7.107438016528926)),
('phase 2', (-103.98955117549275, 50.84303015910711, 2.968416053194016)),
('vanilla_gpt-3.5', (0.26799895454271294, -3.0143348278011217, 2.8418343754397957)),
('phase 2', (-12.654032463669584, 19.877623863346646, 12.288603722274157)),
('vanilla_gpt-3.5', (-113.10410635960515, -17.252563968919645, 16.290724807351513)),
('original code', (-0.3864447086801427, -16.97384066587396, -4.191438763376932)),
('phase 2', (-0.01604524873130037, 0.000720378929505416, -146.54029038859983)),
('vanilla_gpt-3.5', (-30.89068825910931, -58.98785425101215, 5.506072874493928)),
('vanilla_gpt-3.5', (-5.149253731343284, 1.8905472636815919, 18.034825870646767)),
('phase 1', (41.07101815674557, -8.702980157818125, -253.63517965218722)),
('phase 1', (93.56681566364666, 92.89560542304717, 0)),
('phase 2', (99.99909520880159, 99.99918262824104, 0)),
('phase 2', (16.567425569176883, 34.36077057793345, 18.528896672504377)),
('phase 1', (30.9298033772555, 25.919626796462158, 19.535923018990058)),
('phase 1', (99.75950356785859, 99.58655397313962, 44.750522065300224)),
('phase 1', (99.99902769250937, 99.99892756112907, 0.0008435671445036796)),
('phase 2', (12.953946033678232, 16.834043416514504, 0)),
('phase 1', (91.25240900912252, -5.267486094343134, 5.326175245299698)),
('phase 1', (99.84741789380249, 99.84432255274598, 99.84058993558959)),
('vanilla_gpt-3.5', (45.61824729891957, 55.273537986623225, 69.1133596295661)),
('vanilla_gpt-3.5', (-12.009237875288683, -25.92378752886836, 3.983833718244804))]

In [5]:
for idx, y in enumerate(l):
    print(str(idx+1)+"     "+str(y))

1     ('original code', (-6.755319540884875, -3.806391554561063, -2.5205646339875862))
2     ('phase 2', (20.022576548610132, 21.96980386623395, 19.909693805559474))
3     ('phase 2', (10.303461455303776, 10.37739138925505, -1.0322292664895139))
4     ('vanilla_gpt-3.5', (-84.35374149659864, -116.90526957051478, 4.629911666159001))
5     ('phase 1', (82.30084896245637, 82.27837992805941, -22.002491929995813))
6     ('original code', (-10.375, -16.75, -20.78125))
7     ('vanilla_gpt-3.5', (-41.24885215794307, -35.05968778696052, 7.107438016528926))
8     ('phase 2', (-103.98955117549275, 50.84303015910711, 2.968416053194016))
9     ('vanilla_gpt-3.5', (0.26799895454271294, -3.0143348278011217, 2.8418343754397957))
10     ('phase 2', (-12.654032463669584, 19.877623863346646, 12.288603722274157))
11     ('vanilla_gpt-3.5', (-113.10410635960515, -17.252563968919645, 16.290724807351513))
12     ('original code', (-0.3864447086801427, -16.97384066587396, -4.191438763376932))
13     ('phase 2